In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#############Common parameters  ######

In [ ]:
No_Features=10  ### Number of features selected by PCA

In [ ]:
import pandas as pd
import numpy as np
###Load tabular data
LC_RF_PT = pd.read_excel(".../LC_DF_SCT.xlsx" , engine='openpyxl',header=None)   ## Load you Lung Cancer Dataset
HC_RF_PT = pd.read_excel(".../HC_DF_SCT.xlsx" , engine='openpyxl',header=None)   ## Load you Head and Neck Cancer Dataset
Lung_Outcome_all = pd.read_excel(".../LC_outcome.xlsx" , engine='openpyxl',header=None)   ## Load you Lung ccancer Outcomes
Lung_Outcome=Lung_Outcome_all.iloc[:, 2:]

Length_LC_RF_PT= len(LC_RF_PT)
Length_HC_RF_PT = len(HC_RF_PT)
Length_Lung_Outcome = len(Lung_Outcome)



In [ ]:
############### Concatinate Datasets for splitting ##############

In [ ]:
import numpy as np

Concatinated_RF_PT = np.concatenate((LC_RF_PT, HC_RF_PT), axis=0)

In [ ]:
############## Split lung data for five fold cross validation and external testing  ####################

In [ ]:
from sklearn.model_selection import train_test_split

### Split lung patinets from head and neck
Split_LC_RF_PT = SelectedFeatures[0:Length_LC_RF_PT, :]
Split_HC_RF_PT = SelectedFeatures[Length_LC_RF_PT:, :]

X_train_L, X_test_L_EX, Y_train_L, Y_test_L_EX = train_test_split(Split_LC_RF_PT, Lung_Outcome, test_size=0.20)


In [ ]:
############ Normalize datasets  #####################

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_L = scaler.fit_transform(X_train_L)
X_test_L_EX = scaler.transform(X_test_L_EX)


In [ ]:
#############   Reduce feature size by feature selection algorithms o feature extraction algorithms  ###################3

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=No_Features)
X_train_L = pca.fit_transform(X_train_L)
X_test_L_EX = pca.transform(X_test_L_EX)


In [ ]:
############# Split Five folds ###############

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier


# Set random seed for reproducibility
random_seed = 10
np.random.seed(random_seed)

# Convert y_train to a NumPy array (if it's not already)
Y_train_L = np.array(Y_train_L)

# Initialize Random Forest classifier
#rf_classifier = RandomForestClassifier()
rf_classifier = RandomForestClassifier(
    n_estimators=5,
    criterion='gini',
    max_depth=2,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features='auto',
    bootstrap=True,
    random_state=42
)
# Define the number of folds for cross-validation
num_folds = 5
# Initialize KFold with shuffling
kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_seed)

# Initialize an array to store cross-validation scores
cv_scores_MPL_SSL= []
cv_scores_MPL_SL= []
cv_scores_MLP_SSL_EX = []
cv_scores_MLP_SL_EX = []

cv_scores_SVM_SSL= []
cv_scores_SVM_SL= []
cv_scores_SVM_SSL_EX = []
cv_scores_SVM_SL_EX = []

cv_scores_KNN_SSL= []
cv_scores_KNN_SL= []
cv_scores_KNN_SSL_EX = []
cv_scores_KNN_SL_EX = []

cv_scores_Stacking_SSL= []
cv_scores_Stacking_SL= []
cv_scores_Stacking_SSL_EX = []
cv_scores_Stacking_SL_EX = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train_L):
    # Split data into training and validation sets
    X_train_fold, X_val_fold = X_train_L[train_index], X_train_L[val_index]
    y_train_fold, y_val_fold = Y_train_L[train_index], Y_train_L[val_index]

    # Train the classifier
    rf_classifier.fit(X_train_fold, y_train_fold)

    # Label HC datasets_Sudo Labeling
    predictions_HC = rf_classifier.predict(Split_HC_RF_PT)
    # Reshape predictions_HC to match the shape of y_train_fold
    predictions_HC = predictions_HC.reshape(-1, 1)

    # Concatenate predicted HC patients with LC data
    Concatinated_X_RF_PT_Folds = np.concatenate((X_train_fold, Split_HC_RF_PT), axis=0)
    Concatinated_Y_RF_PT_Folds = np.concatenate((y_train_fold, predictions_HC), axis=0)

    ############Classifers Classifers Classifers Classifers Classifers Classifers  Classifers Classifers Classifers Classifers Classifers  ###########################

   ### 1- MLP
    ### Semisupervised
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import roc_auc_score

    mlp_classifier_SSL = MLPClassifier(hidden_layer_sizes=(50, (50//2),2 ), activation='relu', solver='adam', max_iter=100, random_state=10)
    #Internal
    mlp_classifier_SSL.fit(Concatinated_X_RF_PT_Folds, Concatinated_Y_RF_PT_Folds)
    y_predMLP = mlp_classifier_SSL.predict(X_val_fold)
    accuracyMLP_SSL = accuracy_score(y_val_fold, y_predMLP)
    cv_scores_MPL_SSL.append(accuracyMLP_SSL)
    #External
    y_predMLP = mlp_classifier_SSL.predict(X_test_L_EX)
    accuracyMLP_SSL = accuracy_score(Y_test_L_EX, y_predMLP)
    cv_scores_MLP_SSL_EX.append(accuracyMLP_SSL)

    ### Supervised

    mlp_classifier_SL = MLPClassifier(hidden_layer_sizes=(15, (10//2),2 ), activation='relu', solver='adam', max_iter=100, random_state=10)
    #Internal
    mlp_classifier_SL.fit(X_train_fold, y_train_fold)
    y_predMLP = mlp_classifier_SL.predict(X_val_fold)
    accuracyMLP_SL = accuracy_score(y_val_fold, y_predMLP)
    cv_scores_MPL_SL.append(accuracyMLP_SL)
    #External
    y_predMLP = mlp_classifier_SL.predict(X_test_L_EX)
    accuracyMLP_SL = accuracy_score(Y_test_L_EX, y_predMLP)
    cv_scores_MLP_SL_EX.append(accuracyMLP_SL)


############################################################
   ### 2- SVM
    ### Semisupervised
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score

    svm_clf_SSL = SVC(kernel='rbf', C=0.001, gamma='auto', random_state=52)
    #svm_clf_SSL = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

    #Internal
    svm_clf_SSL.fit(Concatinated_X_RF_PT_Folds, Concatinated_Y_RF_PT_Folds)
    SVMy_pred = svm_clf_SSL.predict(X_val_fold)
    accuracySVM_SSL = accuracy_score(y_val_fold, SVMy_pred)
    cv_scores_SVM_SSL.append(accuracySVM_SSL)
    #External
    y_predSVM = svm_clf_SSL.predict(X_test_L_EX)
    accuracySVM_SSL = accuracy_score(Y_test_L_EX, y_predSVM)
    cv_scores_SVM_SSL_EX.append(accuracySVM_SSL)

    ### Supervised
    svm_clf_SL = SVC(kernel='poly', C=1.0, gamma='auto', random_state=110)
    #Internal
    svm_clf_SL.fit(X_train_fold, y_train_fold)
    SVMy_pred1 = svm_clf_SL.predict(X_val_fold)
    accuracySVM_SL = accuracy_score(y_val_fold, SVMy_pred1)
    cv_scores_SVM_SL.append(accuracySVM_SL)
    #External
    y_predSVM2 = svm_clf_SL.predict(X_test_L_EX)
    accuracySVM_SL = accuracy_score(Y_test_L_EX, y_predSVM2)
    cv_scores_SVM_SL_EX.append(accuracySVM_SL)

### 3- KNN
    ### Semisupervised
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score

    KNN_clf_SSL = KNeighborsClassifier(n_neighbors=25)  # Specify the number of neighbors (k)


    #Internal
    KNN_clf_SSL.fit(Concatinated_X_RF_PT_Folds, Concatinated_Y_RF_PT_Folds)
    KNNy_pred = KNN_clf_SSL.predict(X_val_fold)
    accuracyKNN_SSL = accuracy_score(y_val_fold, KNNy_pred)
    cv_scores_KNN_SSL.append(accuracyKNN_SSL)
    #External
    y_predKNN = KNN_clf_SSL.predict(X_test_L_EX)
    accuracyKNN_SSL = accuracy_score(Y_test_L_EX, y_predKNN)
    cv_scores_KNN_SSL_EX.append(accuracyKNN_SSL)

    ### Supervised
    KNN_clf_SL = KNeighborsClassifier(n_neighbors=1)  # Specify the number of neighbors (k)

    #Internal
    KNN_clf_SL.fit(X_train_fold, y_train_fold)
    KNNy_pred = KNN_clf_SL.predict(X_val_fold)
    accuracyKNN_SL = accuracy_score(y_val_fold, KNNy_pred)
    cv_scores_KNN_SL.append(accuracyKNN_SL)
    #External
    y_predKNN = KNN_clf_SL.predict(X_test_L_EX)
    accuracyKNN_SL = accuracy_score(Y_test_L_EX, y_predKNN)
    cv_scores_KNN_SL_EX.append(accuracyKNN_SL)


############################################################
    ### 4- Stacking
    ### Semisupervised
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import StackingClassifier
    from sklearn.linear_model import LogisticRegression

    estimators_SSL = [
    ("mlp_classifier_SSL", mlp_classifier_SSL),
    ("svm_clf_SSL", svm_clf_SSL),
    ("KNN_clf_SSL", KNN_clf_SSL),
    ]


    Stacking_Classifier_SSL = VotingClassifier(estimators_SSL)

    #Internal
    Stacking_Classifier_SSL.fit(Concatinated_X_RF_PT_Folds, Concatinated_Y_RF_PT_Folds)
    y_predStacking = Stacking_Classifier_SSL.predict(X_val_fold)
    accuracyStacking_SSL = accuracy_score(y_val_fold, y_predStacking)
    cv_scores_Stacking_SSL.append(accuracyStacking_SSL)
    #External
    y_predStacking = Stacking_Classifier_SSL.predict(X_test_L_EX)
    accuracyStacking_SSL = accuracy_score(Y_test_L_EX, y_predStacking)
    cv_scores_Stacking_SSL_EX.append(accuracyStacking_SSL)

    ### Supervised

    estimators_SL = [
    ("mlp_classifier_SL", mlp_classifier_SL),
    ("svm_clf_SL", svm_clf_SL),
    ("KNN_clf_SL", KNN_clf_SL),
    ]

    Stacking_Classifier_SL = VotingClassifier(estimators_SL)
    #Internal
    Stacking_Classifier_SL.fit(X_train_fold, y_train_fold)
    y_predStacking = Stacking_Classifier_SL.predict(X_val_fold)
    accuracyStacking_SL = accuracy_score(y_val_fold, y_predStacking)
    cv_scores_Stacking_SL.append(accuracyStacking_SL)
    #External
    y_predStacking = Stacking_Classifier_SL.predict(X_test_L_EX)
    accuracyStacking_SL = accuracy_score(Y_test_L_EX, y_predStacking)
    cv_scores_Stacking_SL_EX.append(accuracyStacking_SL)


############################################################


print("----------------------------------------------------------------------------------")
print("MLP")
print ("    -------  Average cross validation--------")

print("Mean accuracy_MLP_SSL:", [(round(np.mean(cv_scores_MPL_SSL),2)),round(np.std(cv_scores_MPL_SSL),2)])

print("Mean accuracy_MLP_SL:", [round(np.mean(cv_scores_MPL_SL),2),round(np.std(cv_scores_MPL_SL),2)])

print("Mean accuracy_NestedXT_MLP_SSL:", [round(np.mean(cv_scores_MLP_SSL_EX),2),round(np.std(cv_scores_MLP_SSL_EX),2)])

print("Mean accuracy_NestedXT_MLP_SL:", [round(np.mean(cv_scores_MLP_SL_EX),2),round(np.std(cv_scores_MLP_SL_EX),2)])


print("----------------------------------------------------------------------------------")
print("SVM")

print("Mean accuracy_SVM_SSL:", [(round(np.mean(cv_scores_SVM_SSL),2)),round(np.std(cv_scores_SVM_SSL),2)])

print("Mean accuracy_SVM_SL:", [round(np.mean(cv_scores_SVM_SL),2),round(np.std(cv_scores_SVM_SL),2)])

print("Mean accuracy_NestedXT_SVM_SSL:", [round(np.mean(cv_scores_SVM_SSL_EX),2),round(np.std(cv_scores_SVM_SSL_EX),2)])

print("Mean accuracy_NestedXT_SVM_SL:", [round(np.mean(cv_scores_SVM_SL_EX),2),round(np.std(cv_scores_SVM_SL_EX),2)])

print("----------------------------------------------------------------------------------")

print("KNN")

print("Mean accuracy_KNN_SSL:", [(round(np.mean(cv_scores_KNN_SSL),2)),round(np.std(cv_scores_KNN_SSL),2)])

print("Mean accuracy_KNN_SL:", [round(np.mean(cv_scores_KNN_SL),2),round(np.std(cv_scores_KNN_SL),2)])

print("Mean accuracy_NestedXT_KNN_SSL:", [round(np.mean(cv_scores_KNN_SSL_EX),2),round(np.std(cv_scores_KNN_SSL_EX),2)])

print("Mean accuracy_NestedXT_KNN_SL:", [round(np.mean(cv_scores_KNN_SL_EX),2),round(np.std(cv_scores_KNN_SL_EX),2)])

print("Stacking")

print("Mean accuracy_Stacking_SSL:", [(round(np.mean(cv_scores_Stacking_SSL),2)),round(np.std(cv_scores_Stacking_SSL),2)])

print("Mean accuracy_Stacking_SL:", [round(np.mean(cv_scores_Stacking_SL),2),round(np.std(cv_scores_Stacking_SL),2)])

print("Mean accuracy_NestedXT_Stacking_SSL:", [round(np.mean(cv_scores_Stacking_SSL_EX),2),round(np.std(cv_scores_Stacking_SSL_EX),2)])

print("Mean accuracy_NestedXT_Stacking_SL:", [round(np.mean(cv_scores_Stacking_SL_EX),2),round(np.std(cv_scores_Stacking_SL_EX),2)])






In [ ]:
############ Visualize results  #####################

In [ ]:
import pandas as pd
resultDf = pd.read_excel(".../revision-Results.xlsx",sheet_name = "Total",header=0)
new_resultDf = resultDf.iloc[1:,:]
sorted_resultDf = new_resultDf.sort_values("Type")
semi_results = sorted_resultDf.iloc[:6,:]
sup_results = sorted_resultDf.iloc[6:,:]
semi_results


,Feature type,Modalities,Type,FEA,MLP,Unnamed: 5,SVM,Unnamed: 7,KNN,Unnamed: 9,EV,Unnamed: 11
1,HRF,CT,Semi-Supervised,PCA,0.75 (0.06),0.68 (0.03),0.73 (0.07),0.68 (0.0),0.72 (0.06),0.68 (0.01),0.72 (0.06),0.68 (0.0)
3,HRF,PET,Semi-Supervised,PCA,0.77 (0.1),0.72 (0.02),0.76 (0.1),0.72 (0.01),0.76 (0.1),0.72 (0.01),0.73 (0.07),0.68 (0.0)
5,DRF,CT,Semi-Supervised,PCA,0.83 (0.06),0.79 (0.01),0.82 (0.06),0.77 (0.02),0.81 (0.06),0.74 (0.01),0.69 (0.03),0.82 (0.0)
7,DRF,PET,Semi-Supervised,PCA,0.85 (0.05),0.8 (0.01),0.81 (0.06),0.8 (0.01),0.83 (0.06),0.81 (0.04),0.7 (0.05),0.8 (0.0)
9,DRF+HRF,PET,Semi-Supervised,PCA,0.71 (0.08),0.75 (0.0),0.71 (0.08),0.75 (0.0),0.7 (0.07),0.75 (0.0),0.73 (0.05),0.68 (0.0)
11,DRF+HRF,CT,Semi-Supervised,PCA,0.7 (0.04),0.82 (0.0),0.69 (0.03),0.82 (0.0),0.69 (0.03),0.82 (0.0),0.69 (0.03),0.82 (0.0)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Setting bold font globally
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'

a=0

for i,row in sup_results.iterrows():

  # Sample data for plotting or other operations
  stores = ['MLP', 'SVM', 'KNN','EV']

  sales_2019 = [float(row['MLP'].split("(")[0]),
                float(row['SVM'].split("(")[0]),
                float(row['KNN'].split("(")[0]),
                float(row['EV'].split("(")[0])]
  std_2019 = [float(row['MLP'].split("(")[1].split(")")[0]),
                float(row['SVM'].split("(")[1].split(")")[0]),
                float(row['KNN'].split("(")[1].split(")")[0]),
                float(row['EV'].split("(")[1].split(")")[0])]

  # print(semi_results.iloc[a,5])
  # Standard deviations for the sample data
  sales_2020 = [float(semi_results.iloc[a,4].split("(")[0]),
                float(semi_results.iloc[a,6].split("(")[0]),
                float(semi_results.iloc[a,8].split("(")[0]),
                float(semi_results.iloc[a,10].split("(")[0])]
  std_2020 = [float(semi_results.iloc[a,4].split("(")[1].split(")")[0]),
                float(semi_results.iloc[a,6].split("(")[1].split(")")[0]),
                float(semi_results.iloc[a,8].split("(")[1].split(")")[0]),
                float(semi_results.iloc[a,10].split("(")[1].split(")")[0])]

  a =a+1
  # Create an index for each tick position
  x = np.arange(len(stores))

  # Bar width
  width = 0.40

  # Creating the bars with error bars
  fig, ax = plt.subplots()
  bars1 = ax.bar(x - width/2, sales_2019, width, label='Semi-supervised', yerr=std_2019, capsize=2, color='red')
  bars2 = ax.bar(x + width/2, sales_2020, width, label='Supervised', yerr=std_2020, capsize=2, color='green')

  # Adding some text for labels, title, and custom x-axis tick labels, etc.
  ax.set_ylabel('Accuracy', fontsize=20)  # Increased font size for y-axis label
  ax.set_title(row['Feature type']+"-"+row['Modalities'], fontsize=20)  # Increased title font size for consistency
  ax.set_xticks(x)
  ax.set_xticklabels(stores, fontsize=16, fontweight='bold')  # Maintaining increased font size for x-tick labels

  # Set y-axis limits
  ax.set_ylim(0, 1)

  # Increase font size of y-axis tick labels
  ax.tick_params(axis='y', labelsize=14)  # Set font size for y-axis tick labels

  # ax.legend()

  # Adding grid
  ax.grid(True, linestyle='--', which='major', color='grey', alpha=0.5)

  # Show the plot
  plt.show()
  fig.savefig(row['Feature type']+"_"+row['Modalities']+'.png')